#TAG SMART EMAILS

GENSIM AND LDA

https://radimrehurek.com/gensim/apiref.html

In [1]:
#Library
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import spacy
import pandas as pd

In [2]:
%pwd

'C:\\Users\\HomePC\\tagger-ds'

In [4]:
import requests
url = 'https://raw.githubusercontent.com/Moly-malibu/Tag-Smart-Emails/master/Enronemails.csv?token=AN2RW2GO4AWTXK4OPUDXJV26ZQ3QC'
myfile = requests.get(url)
open('email.csv', 'wb').write(myfile.content)

24004915

In [6]:
#EMAILS FROM ENRON COMPANY SAMPLING
df = pd.read_csv("email.csv")
print(df.shape) 
df.head(7).drop(['Unnamed: 0'], axis=1)

(10000, 3)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...


In [ ]:
#TO GET EMAIL BODY
[s.strip() for s in df['message'][1].splitlines()]

['Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>',
 'Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)',
 'From: phillip.allen@enron.com',
 'To: john.lavorato@enron.com',
 'Subject: Re:',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Phillip K Allen',
 'X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>',
 'X-cc:',
 'X-bcc:',
 "X-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail",
 'X-Origin: Allen-P',
 'X-FileName: pallen (Non-Privileged).pst',
 '',
 'Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.',
 '',
 'As far as the business meetings, I think it would be more productive to try and stimulate discussions across

In [ ]:
' '.join([s.strip() for s in df['message'][1].splitlines()][15:])

" Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.  As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  My suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time."

In [ ]:
#CLEAN EMAIL BODY
def cleanemail(email):
    return ' '.join([s.strip() for s in email.splitlines()][15:])

df['email_body'] = df['message'].apply(cleanemail)

df.head()

,Unnamed: 0,file,message,email_body
0,0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast
1,1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes th...
2,2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!
3,3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy, Can you send me a schedule of the sal..."
4,4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.


In [ ]:
#CLEAN HTML
import re
import html

def cleanup_html(raw_html):
    clean_html = re.sub(r'<.*?>', '', raw_html)
    clean_html_http = re.sub(r'http\S+([\.]{3})?', '', clean_html)
    clean_txt = html.unescape(clean_html)
    return clean_txt
df['email_body'] = df['email_body'].apply(cleanup_html)
df.sample(10)
for row in df['email_body'].sample(10):
    print(row)
    print()

 Rhonda,  You need to check with Genia as I have never handled the physical power agreement matters.  Susan  -----Original Message----- From: 	Denton, Rhonda L. Sent:	Tuesday, January 15, 2002 2:17 PM To:	Bailey, Susan Cc:	Hansen, Leslie Subject:	Southern Company Netting  Here's Southern.  I never received a copy of the Virginia Electric Master Netting.  We do have netting within the EEI. >

 10

 are you free for a drink/dinner Wednesday night?

 John:  I would like to take a vacation day this Thursday.  I have to go to court and am having some work done to my house.  I will still logon from home and can be reached by cell.   -Ina

 please add mike grigsby to distribution list.

 Why are his requests coming to me?

 Rahil: I have never commented favorably nor unfavorably about Kase's newsletter.   I think publishing independent market evaluations could be beneficial.  The more interesting content that is published, the better.      From:  Rahil Jafry 08/28/2000 07:48 PM      To: John 

In [ ]:
#CLEAN DATA
#remove whitespace
df['email_body'] = df['email_body'].str.strip().str.lower()

#Start with date
df['email_body'].str.match('\d?\d/\d?\d/\d{4}').all()

#Replace all non-overlapping matches
df['email_body'] = df['email_body'].str.replace('[^a-zA-Z\s]', '').str.replace('\s+', ' ')

In [ ]:
import nltk
import string
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Getting Stopwords
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = stopwords.words('english')
sno = SnowballStemmer('english')
print (stop)
print('***************************************')
print(sno.stem('tasty'))

STOP_WORDS = list(STOP_WORDS)
print('\n'.join([word for word in STOP_WORDS[:10]]))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
tokens = []
for doc in nlp.pipe(df['email_body'], batch_size=500):

    doc_tokens = []

    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False):
            doc_tokens.append(token.lemma_.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [ ]:
df['tokens'].head()

0                                           [forecast]
1    [travel, business, meeting, take, fun, trip, e...
2                              [test, successful, way]
3    [randy, send, schedule, salary, level, schedul...
4                                [let, shoot, tuesday]
Name: tokens, dtype: object

In [ ]:
id2word = Dictionary(df['tokens'])
print(len(id2word))

45828


In [ ]:
id2word.filter_extremes(no_below=5, no_above=.98)
print(len(id2word))

14979


In [ ]:
#https://radimrehurek.com/gensim/corpora/dictionary.html

corpus = [id2word.doc2bow(d) for d in df['tokens']]

In [ ]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )

In [ ]:
model = LdaMulticore(corpus=corpus, num_topics=10, id2word=id2word, workers=12, passes=5)

In [ ]:
import re

words = [re.findall(r'"([^"]*)"',t[1]) for t in model.print_topics()]

In [ ]:
topics = [' '.join(t[0:5]) for t in words]

In [ ]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n")

------ Topic 0 ------
say power enron company energy
------ Topic 1 ------
enron company say market share
------ Topic 2 ------
enron john subject phillip email
------ Topic 3 ------
gas subject john cc think
------ Topic 4 ------
pm send subject message original
------ Topic 5 ------
pm subject john cc enron
------ Topic 6 ------
power phillip cost iso market
------ Topic 7 ------
image email company price offer
------ Topic 8 ------
pm message john wine price
------ Topic 9 ------
email john subject cc gas


In [ ]:
!pip install pyLDAvis
import pyLDAvis

     |████████████████████████████████| 1.6MB 12.4MB/s eta 0:00:01
     |████████████████████████████████| 552kB 48.3MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=bda091ceb4a865643e0992b50d678b7d1dfb046dcba917c84764c5fa50b75beb
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=f15a85d6c1aa91b7853a14af6d36c46602056ab849032c1f344d52df1ab17935
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.097480  0.029237       1        1  15.653592
6      0.064118  0.024697       2        1  11.354891
5     -0.083509  0.045605       3        1  11.346098
3      0.040539 -0.107909       4        1  10.477987
0      0.090415  0.021751       5        1  10.465056
7      0.034947  0.001972       6        1  10.435542
4     -0.083777  0.045856       7        1   9.696163
2     -0.038861  0.024072       8        1   9.091611
8     -0.102796 -0.058269       9        1   6.103777
9     -0.018555 -0.027013      10        1   5.375285, topic_info=             Term          Freq         Total Category  logprob  loglift
2682        image   3534.000000   3534.000000  Default  30.0000  30.0000
173            pm   8667.000000   8667.000000  Default  29.0000  29.0000
208         enron  11895.000000  11895.000000  Default  28.0000  28.0000
962         email   5805.000000   5805.000000  Default  27.0000  27.0000
77           john   7465.000000   7465.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
348        market    218.338638   5414.987793  Topic10  -5.6795  -0.2875
124           day    170.389771   2867.989258  Topic10  -5.9275   0.1001
57              s    184.253159   3863.604736  Topic10  -5.8493  -0.1197
37          think    157.403336   2512.243164  Topic10  -6.0068   0.1532
502   information    150.718857   2717.434570  Topic10  -6.0502   0.0313

[931 rows x 6 columns], token_table=      Topic      Freq Term
term                      
90        1  0.036121     
90        2  0.109185     
90        3  0.180607     
90        4  0.126425     
90        5  0.021344     
...     ...       ...  ...
6314      6  0.738625  zip
6314      7  0.026379  zip
6314      8  0.017586  zip
6314      9  0.096725  zip
6314     10  0.017586  zip

[4694 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 6, 4, 1, 8, 5, 3, 9, 10])

In [ ]:
#'corpus'

distro = [lda[d] for d in corpus]

num_topics = 20

def update(doc):
        d_dist = {k:0 for k in range(0,num_topics)}
        for topic in doc:
            d_dist[topic[0]] = topic[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [ ]:
#Correlated
topics = pd.DataFrame.from_records(new_distro)
topics.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.533333,0,0,0,0,0
1,0.000000,0.000000,0.000000,0.102725,0.000000,0.000000,0.000000,0.000000,0.882831,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
2,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.766666,0.016667,0.016667,0.016667,0.016667,0,0,0,0,0
3,0.000000,0.000000,0.637557,0.308276,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
4,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.766666,0.016667,0.016667,0,0,0,0,0


In [ ]:
df['email_body'] = topics.idxmax(axis=1)

In [ ]:
#Topic  
df['email_body'].describe()

count    10000.000000
mean         7.249000
std          4.349539
min          0.000000
25%          3.000000
50%          8.000000
75%         12.000000
max         14.000000
Name: email_body, dtype: float64

In [ ]:
df['email_body'].value_counts()

12    2353
1     1149
3     1031
8      762
10     723
9      651
6      631
5      533
2      488
13     354
0      350
7      311
14     281
4      218
11     165
Name: email_body, dtype: int64

In [ ]:
def save_data_frame_as_csv(df=None, file=None):
    """
    Save a Pandas DataFrame to CSV format.
    """
    try:
        if not file.endswith('.csv'):
            file += '.csv'
        df.to_csv(file, index=False)
        print(f'DataFrame saved as: {file}')
    except Exception as e:
        print("DataFrame couldn't be saved: ", sys.exc_info()[0])
        raise

In [ ]:
save_data_frame_as_csv(df, './merged_data.csv')

In [ ]:
def save_data_frame_as_json(df=None, file=None, orient='records'):
  """
  Save a Pandas DataFrame to Json format.
  """
  try:
    if not file.endswith('.json'):
      file += '.json'
    df.to_json(file, orient=orient)
    print(f'Dataframe saved as: {file}')
  except Exception as e:
    print("Dataframe couldn't be saved: ", sys.exc_info()[0])
    raise

In [ ]:
save_data_frame_as_json(df, '.merged_data.json', orient='records')